In [1]:
import numpy as np

### Instantiate the prediction task

In [2]:
from jenga.tasks.income import IncomeEstimationTask

task = IncomeEstimationTask(seed=42)

### Task details

In this task, we try to predict the income level of a person (more or less than 50K dollars per years) from demographic and work-related data. It is often used as a proxy to study automated decision making for loan applications.

The original data is available in the UCI machine learning repository at https://archive.ics.uci.edu/ml/datasets/adult

In [3]:
task.train_data

,workclass,occupation,marital_status,education,hours_per_week,age
6836,Private,Other-service,Never-married,Some-college,30,23
24226,Private,Prof-specialty,Never-married,Masters,40,45
25154,Private,Craft-repair,Married-civ-spouse,Assoc-voc,50,35
22509,Private,Tech-support,Never-married,Bachelors,15,27
18759,Local-gov,Other-service,Never-married,HS-grad,40,25
...,...,...,...,...,...,...
24339,Private,Exec-managerial,Married-civ-spouse,Some-college,50,43
29239,Private,Machine-op-inspct,Married-civ-spouse,7th-8th,40,48
28248,Local-gov,Other-service,Never-married,1st-4th,20,46
28880,Private,Sales,Married-civ-spouse,HS-grad,60,53


### Train the provided baseline model

Jenga allows us to easily train and evaluate a logistic regression model for this task. Have a look at https://github.com/schelterlabs/jenga/blob/master/jenga/tasks/income.py if you want to know the details.


In [4]:
model = task.fit_baseline_model()

In [5]:
f"The ROC AUC score on the test data is {task.get_baseline_performance()}"

'The ROC AUC score on the test data is 0.8796088300227616'

### Let's have a look at the test data

In [6]:
task.test_data

,workclass,occupation,marital_status,education,hours_per_week,age
26203,Private,Transport-moving,Married-civ-spouse,10th,60,50
19843,Private,Adm-clerical,Never-married,Masters,40,40
12610,Private,Adm-clerical,Divorced,Masters,25,42
6531,Private,Adm-clerical,Widowed,7th-8th,35,31
9390,Private,Craft-repair,Married-civ-spouse,5th-6th,40,60
...,...,...,...,...,...,...
23767,Private,Exec-managerial,Never-married,HS-grad,40,33
27214,Private,Craft-repair,Married-civ-spouse,Bachelors,36,30
20071,Private,Sales,Separated,Bachelors,50,33
2401,Private,Machine-op-inspct,Never-married,Some-college,40,20


### Corruptions

Jenga provides a set of predefined data corruptions that we can use to simulate errors in the test data. We will simulate 'implicit missing values' in a column and see how this impacts the prediction quality.

Jenga supports much more predefined corruptions, have a look at https://github.com/schelterlabs/jenga/tree/master/jenga/corruptions.


In [7]:
from jenga.corruptions.generic import MissingValues

marital_status_corruption = MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN')

In [8]:
corrupted_test_data = marital_status_corruption.transform(task.test_data)
corrupted_test_data

,workclass,occupation,marital_status,education,hours_per_week,age
26203,Private,Transport-moving,BROKEN,10th,60,50
19843,Private,Adm-clerical,BROKEN,Masters,40,40
12610,Private,Adm-clerical,BROKEN,Masters,25,42
6531,Private,Adm-clerical,BROKEN,7th-8th,35,31
9390,Private,Craft-repair,BROKEN,5th-6th,40,60
...,...,...,...,...,...,...
23767,Private,Exec-managerial,BROKEN,HS-grad,40,33
27214,Private,Craft-repair,BROKEN,Bachelors,36,30
20071,Private,Sales,BROKEN,Bachelors,50,33
2401,Private,Machine-op-inspct,BROKEN,Some-college,40,20


In [9]:
y_pred = model.predict_proba(corrupted_test_data)

f"The ROC AUC score on the corrupted test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the corrupted test data is 0.7995970837525652'

### Jenga's evaluators

Jenga provides a set of evaluators which allow us to automate the evaluation of the impact of given data corruptions.

In [10]:
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

evaluator = CorruptionImpactEvaluator(task)

In [11]:
corruptions = [
    MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN'),
    MissingValues(column='age', fraction=0.05, na_value=-999),
]

### Run the evaluation of the corruptions with 10 repetitions 

In [12]:
num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, *corruptions)

0/20 (0.024960000000000093)
10/20 (0.2159760000000004)


### Investigate the impact on the predictive performance of the model

In [13]:
for validation_result in results:
    
    print(validation_result.corruption)
    print(f"""
     Score (AUC) on 
      clean data:     {validation_result.baseline_score}
      corrupted data: {np.mean(validation_result.corrupted_scores)}
     """)    
    print("\n")

MissingValues: {'column': 'marital_status', 'fraction': 0.99, 'sampling': 'MCAR', 'na_value': 'BROKEN'}

     Score (AUC) on 
      clean data:     0.8796088300227616
      corrupted data: 0.7992445919726218
     


MissingValues: {'column': 'age', 'fraction': 0.05, 'sampling': 'MCAR', 'na_value': -999}

     Score (AUC) on 
      clean data:     0.8796088300227616
      corrupted data: 0.8441495695194139
     


